In [4]:
import tensorflow as tf

In [2]:
from sklearn.datasets import fetch_20newsgroups
data= fetch_20newsgroups(subset='all')

In [38]:
emb_dim=50
vocab_size=10000
num_classes=2
tf.reset_default_graph()

word_inds=tf.placeholder(tf.int32, [None], name='word_inds')
true_classes=tf.placeholder(tf.int32, name='true_classes')

word_matrix=tf.get_variable("word_matrix", [vocab_size, emb_dim],initializer=tf.random_normal_initializer(0,0.1))
emb=tf.nn.embedding_lookup(word_matrix, word_inds)

weights=tf.Variable(tf.random_normal([emb_dim, num_classes]))
biases=tf.Variable(tf.random_normal([num_classes]))
pred=tf.reduce_mean(tf.matmul(emb,weights)+biases, axis=0)

oh_true_classes=tf.one_hot(true_classes, num_classes)
loss_val = tf.nn.softmax_cross_entropy_with_logits(labels=oh_true_classes, logits=pred)
train_op=tf.train.AdamOptimizer().minimize(loss_val)

In [14]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [32]:
cats = ['alt.atheism', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train', categories=cats)
newsgroups_train.target.shape

(1073,)

In [33]:
tokenizer=Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(newsgroups_train.data)
sequences=tokenizer.texts_to_sequences(newsgroups_train.data)
pad_seqs=pad_sequences(sequences)
pad_seqs.shape

(1073, 6189)

In [40]:
num_epoch=3

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(num_epoch):
        mean_loss_val=0
        for i in range(pad_seqs.shape[0]):
            pad_seq=pad_seqs[i,:]
            text_target=newsgroups_train.target[i]
            
            a, _loss_val=sess.run([train_op, loss_val], 
                                  feed_dict={word_inds:pad_seq, true_classes:text_target})
            mean_loss_val+=_loss_val
        print(mean_loss_val/pad_seqs.shape[0])

0.682467439683
0.643626513361
0.608084286991


In [37]:
newsgroups_train.target[1]

1